In [7]:
import numpy as np
import pandas as pd
import math

In [8]:
def round_to_nearest(value):
    
    difference_ceil = math.ceil(value) - value
    if difference_ceil <= 0.5:
        return math.floor(value)
    else:
        return math.ceil(value)

In [9]:
# providing patient id + corrsponding matching post code data

# Patient unique = 1072
Patient_CAP = pd.read_csv('.../Patient_CAP.csv')
Patient_CAP.dropna(subset = ["CAP"], axis = 0, inplace = True)
Patient_CAP["CAP"] = Patient_CAP["CAP"].astype(int)
Patient_CAP["CAP"] = Patient_CAP["CAP"].astype(str)

# Postcode unique = 213206
Path = '.../postcode_of_patient.csv'
Patient_postcode = pd.read_csv(Path)

# Why left? as we only need the airpollution for positcode in Mondino dataset
matched_postcode = pd.merge(Patient_CAP, Patient_postcode, 
                            left_on=["CAP"],right_on =["postcode"], 
                            how='left')

# Select only postcodes in Italy
postcode_Italy = matched_postcode[matched_postcode["country"]==2.0]

# matching_post_code = It means the matched station postcode for each patient postcode
# postcode = patient postcode
# Unique patient id = 1069 , patient postcode = 261, matching station postcode = 108
# this data is patient id and its postcode associated with matching station postcode
postcode_mondino = postcode_Italy[["Paziente ID", "postcode", "matching_post_code"]]
postcode_mondino.head()

#postcode_Italy.to_csv("postcode_Italy.csv", index = False)

,Paziente ID,postcode,matching_post_code
0,2774,70043,901.0
1,3936,72100,853.0
2,5835,70025,1046.0
4,9988,27100,669.0
5,10096,15048,602.0


In [10]:
# Weather data with patient id in Italy
path = ".../Italy_updated.csv"
weather_data_Italy = pd.read_csv(path)

#float64 to int
postcode_mondino["matching_post_code"] = postcode_mondino["matching_post_code"].astype(int)
# int64 to int32
weather_data_Italy["postcode"] = weather_data_Italy["postcode"].astype(int)

# Getting the weather data for each postcode in postcode_Italy data
# We only need the weather data for mondino patients
weather_data = pd.merge(postcode_mondino, weather_data_Italy, 
                        left_on=["matching_post_code"],right_on =["postcode"], 
                        how='left')

# Selecting necessary columns
weather_data = weather_data[["Paziente ID","postcode_x", 
                             "matching_post_code",
                             "measurement",
                             "timestamp",
                             "param"]]

weather_data.rename(columns={"postcode_x": "patient_postcode", 
                             "matching_post_code": "matching_station_post_code"}, 
                    inplace = True) 
# patient id = 1069, patient_postcode = 261, matching_station_post_code = 108

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
# Merged Airpollution data with Fs-score data 

# unique FS_score patients = 919 
FS_Scores_data = pd.read_csv('.../imputed_data_EWMA.csv')
# unique hashed patient id = 814
Airquality_data = pd.read_csv(".../Updated_Airquality.csv")

# Doing some preprocessing for both data
# Remove duplicate rows in clinical data
FS_Scores_data.drop_duplicates(subset=["Patient_ID", "Date_of_visit"],
                          keep = "first",
                          inplace=True, 
                          ignore_index=True)

# Convert object features to date feature preparing for merge
Airquality_data['start_date'] = pd.to_datetime(Airquality_data['start_date'])
Airquality_data['end_date'] = pd.to_datetime(Airquality_data['end_date'])
Airquality_data['patient_id']=Airquality_data['patient_id'].astype(object)

FS_Scores_data['Date_of_visit'] = pd.to_datetime(FS_Scores_data['Date_of_visit'])
FS_Scores_data['Patient_ID']=FS_Scores_data['Patient_ID'].astype(object)

# We only need air pollution data from air quality data (after 26 there are clinical missing data)
# Unique hashed patient id = 814
Airquality_data = Airquality_data.iloc[:,0:26] # size = (242414, 26)

# round imputed clinical data
# unique patient id = 919
FS_Scores_data.iloc[:,2:10] = FS_Scores_data.iloc[:,2:10].applymap(round_to_nearest)

# As we have air pollution data after 2013, so, wee need fs-score data after 2013 in order to merge
# unique patient id = 630
Data_after_2013 = FS_Scores_data[FS_Scores_data['Date_of_visit'] >= "2013-01-01"] # size = (4491, 14)
Data_after_2013.shape 

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(4491, 14)

In [109]:
# This celle try to select air pollution data from 4-24-48 weeks ago for each patient
# For pollutant_mean feature, mean value should be measured and for pollutant_who, sum value should be considered

selected_feature_mean = ['PM25_mean', 'PM10_mean', 'CO_mean', 'NO2_mean', 'O3_mean', 'SO2_mean',
                         'wind_speed_mean', 'humidity_mean', 'sealevel_pressure_mean',
                         'global_radiation_mean', 'precipitation_mean',
                         'average_temperature_mean', 'min_temperature_mean',
                         'max_temperature_mean']

selected_feature_who = ['PM25_who', 'PM10_who', 'CO_who', 'NO2_who', 'O3_who', 'SO2_who']


matched_data = []

for index, row in Data_after_2013.iterrows():
    patient_id = row['Patient_ID']
    visit_date = row['Date_of_visit']
    
    if patient_id in list(Airquality_data["patient_id"]):
        
        matching_rows = Airquality_data[(Airquality_data['patient_id'] == patient_id) & 
                                        (Airquality_data['start_date'] <= visit_date) & 
                                        (Airquality_data['end_date'] >= visit_date - pd.DateOffset(weeks=48))]
        
        mean_air_quality = matching_rows[selected_feature_mean].mean(skipna=True).round(3)
        who_air_quality = matching_rows[selected_feature_who].sum(skipna=True)


        matched_row = Airquality_data[(Airquality_data['patient_id'] == patient_id) & 
                                      (Airquality_data['start_date'] <= visit_date) & 
                                      (Airquality_data['end_date'] >= visit_date)]
        if len(matched_row) != 1:
            print(index, patient_id, visit_date, matched_row.shape)
        
        elif len(matched_row) >= 1:
            
            merged_row = {
                "Hashed_patient_id": matching_rows.iloc[0]["hashed_patient_id"],
                'Patient_id': patient_id,
                'Date_of_visit': visit_date,
                "Start_date": matched_row.iloc[0]["start_date"],
                "End_date": matched_row.iloc[0]["end_date"],
                "Start_date_4WeeksAgo": matching_rows.iloc[0]["start_date"],
                "end_data_4WeeksAgo": matching_rows.iloc[-1]["end_date"],
                'PM25_mean':mean_air_quality[0],
                'PM25_who': who_air_quality[0],
                'PM10_mean':mean_air_quality[1],
                'PM10_who':who_air_quality[1],
                'CO_mean':mean_air_quality[2],
                'CO_who':who_air_quality[2],
                'NO2_mean':mean_air_quality[3],
                'NO2_who':who_air_quality[3],
                'O3_mean':mean_air_quality[4],
                'O3_who':who_air_quality[4],
                'SO2_mean':mean_air_quality[5],
                'SO2_who':who_air_quality[5],
                'wind_speed_mean':mean_air_quality[6],
                'humidity_mean':mean_air_quality[7],
                'sealevel_pressure_mean':mean_air_quality[8],
                'global_radiation_mean':mean_air_quality[9],
                'precipitation_mean':mean_air_quality[10],
                'average_temperature_mean':mean_air_quality[11],
                'min_temperature_mean':mean_air_quality[12],
                'max_temperature_mean':mean_air_quality[13],
            }
            matched_data.append(merged_row)
            
        else:
            continue

    else:
        continue

# The reason why we have 1104 missing data in weather data of matched_data is 
# that we do not have weather data for patient id in FS-Score data
# The reason why we have less data compare to clinical data is that, we do not have enough air pollution data
# for each specific patient in spacific  date of visit
matched_data = pd.DataFrame(matched_data) # Unique patient id = 308 , size = 2477 * 25 


300 5835 2018-11-10 00:00:00 (0, 26)
480 27391 2017-11-10 00:00:00 (0, 26)
604 37048 2015-10-02 00:00:00 (0, 26)
605 37048 2016-01-26 00:00:00 (0, 26)
606 37048 2016-08-25 00:00:00 (0, 26)
748 37057 2017-09-01 00:00:00 (0, 26)
1182 37087 2019-11-02 00:00:00 (0, 26)
1358 37098 2017-06-02 00:00:00 (0, 26)
1789 37128 2019-08-07 00:00:00 (0, 26)
1967 37150 2017-08-05 00:00:00 (0, 26)
2256 37180 2017-06-03 00:00:00 (0, 26)
2397 37191 2017-11-05 00:00:00 (0, 26)
2547 37205 2017-08-03 00:00:00 (0, 26)
2663 37211 2016-09-05 00:00:00 (0, 26)
2884 37227 2017-06-03 00:00:00 (0, 26)
2894 37228 2017-10-07 00:00:00 (0, 26)
2916 37229 2016-12-05 00:00:00 (0, 26)
2986 37232 2014-06-05 00:00:00 (0, 26)
3005 37234 2017-09-06 00:00:00 (0, 26)
3526 37272 2014-11-02 00:00:00 (0, 26)
3691 37282 2019-07-05 00:00:00 (0, 26)
3810 37289 2017-10-02 00:00:00 (0, 26)
3993 37298 2017-03-02 00:00:00 (0, 26)
4524 37346 2013-12-11 00:00:00 (0, 26)
4713 37365 2017-11-04 00:00:00 (0, 26)
4947 37380 2016-09-05 00:00:00 (

In [112]:
# Merged Air pollution and Fs-score data after 2013
final_data = pd.merge(Data_after_2013, matched_data, 
                      left_on=['Patient_ID','Date_of_visit'],
                      right_on =['Patient_id', 'Date_of_visit'], 
                      how='inner')

# Selecting necessary features
final_data = final_data[['Hashed_patient_id','Patient_id','Date_of_visit','Start_date', 'End_date',
                         "Start_date_4WeeksAgo","end_data_4WeeksAgo",
                         'EDSS_score_assessed_by_clinician','Sex','Age','MS.in.pediatric.age',
                         'Pyramidal','Cerebellar','Thronchioencephalic', 'Sensitive', 'Sphincteric',
                         'Visual', 'Mental','Deambulation','PM25_mean', 'PM25_who',
                         'PM10_mean', 'PM10_who', 'CO_mean', 'CO_who', 'NO2_mean', 'NO2_who',
                         'O3_mean', 'O3_who', 'SO2_mean', 'SO2_who', 'wind_speed_mean',
                         'humidity_mean', 'sealevel_pressure_mean', 'global_radiation_mean',
                         'precipitation_mean', 'average_temperature_mean',
                         'min_temperature_mean', 'max_temperature_mean']]

final_data.shape # Unique patient id = 308, size = 2487 * 37
final_data.to_csv("48Weeks_Merged_AirFS.csv",index=False)


(2472, 39)

In [93]:
# As we have missing weather data final integrated data, so, we should impute them from weather data extracted between 2011-2022
column_to_param = {
    'wind_speed_mean': 8.0,
    'humidity_mean': 9.0,
    'sealevel_pressure_mean': 10.0,
    'global_radiation_mean': 11.0,
    'precipitation_mean': 12.0,
    'average_temperature_mean': 13.0,
    'min_temperature_mean': 14.0,
    'max_temperature_mean': 15.0
}

final_data['Date_of_visit'] = pd.to_datetime(final_data['Date_of_visit'])
weather_data['timestamp'] = pd.to_datetime(weather_data['timestamp'])

def fill_missing_value(row, column_to_param, weather_data):
    for column, param_value in column_to_param.items():
        if pd.isna(row[column]):
            matching_row = weather_data[
                (weather_data["Paziente ID"] == row["Patient_ID"]) &
                (weather_data["timestamp"] == row["Date_of_visit"]) &
                (weather_data["param"] == param_value)]
            
            if not matching_row.empty:
                row[column] = matching_row["measurement"].values[0]
    return row


result = final_data.apply(lambda row: fill_missing_value(row, column_to_param, weather_data), axis=1)
